In [1]:
import os
import keras
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2024-04-25 19:02:41.415832: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 19:02:41.538095: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 19:02:41.538148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 19:02:41.550985: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 19:02:41.582746: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 19:02:41.583767: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
from functools import partial
def model():
    Conv2D = partial(
        keras.layers.Conv2D,
        activation=keras.activations.relu,
        padding   ='same',
    )
    model = keras.models.Sequential(
        layers=[
            Conv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
            keras.layers.MaxPooling2D(2),
            Conv2D(filters=128,kernel_size=3),
            Conv2D(filters=128,kernel_size=3),
            keras.layers.MaxPooling2D(2),
            Conv2D(filters=256,kernel_size=3),
            Conv2D(filters=256,kernel_size=3),
            keras.layers.MaxPooling2D(2),
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation=keras.activations.relu),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(64, activation=keras.activations.relu),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10, activation=keras.activations.softmax)
        ]
    )
    return model
    

In [3]:
model = model()

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        3200      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 256)         2

In [5]:
fashionMnist = keras.datasets.fashion_mnist
( xtrainFull, ytrainFull ), ( xtestFull, ytestFull ) = fashionMnist.load_data()

In [6]:
xtestV, xtrain = xtrainFull[:5000]/255.0 , xtrainFull[5000:]/255.0
ytestV, ytrain = ytrainFull[:5000] , ytrainFull[5000:]

In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss     =keras.losses.sparse_categorical_crossentropy
)

model.fit(xtrain, ytrain, batch_size=32, epochs=3)

Epoch 1/3
1719/1719 [==============================] - 309s 178ms/step - loss: 0.7562
Epoch 2/3
1719/1719 [==============================] - 298s 173ms/step - loss: 0.4325
Epoch 3/3
1719/1719 [==============================] - 301s 175ms/step - loss: 0.3670


In [8]:
import numpy as np
pred = np.argmax(model.predict(xtestV), axis=-1)

157/157 [==============================] - 8s 48ms/step


In [9]:
from sklearn.metrics import classification_report

In [10]:
print(classification_report(ytestV, pred))

              precision    recall  f1-score   support

           0       0.79      0.89      0.84       457
           1       0.99      0.99      0.99       556
           2       0.78      0.88      0.83       504
           3       0.90      0.93      0.91       501
           4       0.88      0.71      0.79       488
           5       0.99      0.98      0.98       493
           6       0.72      0.63      0.67       493
           7       0.99      0.95      0.97       512
           8       0.98      0.98      0.98       490
           9       0.94      0.99      0.97       506

    accuracy                           0.90      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.90      0.90      0.89      5000

